Assignment 4
Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from assets/wikipedia_data.html and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in assets/nfl.csv), MLB (baseball, in assets/mlb.csv), NBA (basketball, in assets/nba.csv or NHL (hockey, in assets/nhl.csv). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: what is the win/loss ratio's correlation with the population of the city it is in? Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with pearsonr, so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%*4=80%) of the grade for this assignment. You should only use data from year 2018 for your analysis -- this is important!

Notes
Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NHL using 2018 data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def get_area(team):
    for each in list(nhl_cities.index.values):
        if team in each:
            return nhl_cities.loc[each, 'Metropolitan area']


nhl_df = pd.read_csv("nhl.csv")
cities = pd.read_html("wikipedia_data.html")[1] #REVISAR EL PORQUÉ DEL [1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0) # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
population
# cities['NHL'] = cities['NHL'].apply(lambda x: clear_data(x))
cities['NHL'] = cities['NHL'].apply(lambda x: (re.split("[[]",x)[0])) #CLEANING
nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
nhl_cities = nhl_cities.drop(['—', ''], axis=0)
nhl_cities  #len 28

# nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
# nhl_df['team']

nhl_df['team'] = nhl_df['team'].apply(lambda x: (re.split("[*]",x)[0]))#CLEANING
nhl_df['team']
nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1]) 
nhl_df['area']

nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x)) #RELACION DE EQUIPOS CON ÁREAS METROPOLITANAS
nhl_df


out = []
for group, frame in nhl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = (total_wins / total_matches)
    out_dict = {
        'Area': group,
        'W': total_wins,
        'L': total_losses,
        'TM': total_matches,
        'Ratio': ratio
    }
    out.append(out_dict)

new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
new_df
out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])
out_df
population_by_region = []  # pass in metropolitan area population from cities
win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
population_by_region = out_df['Population (2016 est.)[8]']
win_loss_by_region = out_df['Ratio']
corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
corr
len(population_by_region)
# win_loss_by_region

<ipython-input-1-0dd8c28bb22f>:20: FutureWarning: Possible nested set at position 1
  cities['NHL'] = cities['NHL'].apply(lambda x: (re.split("[[]",x)[0])) #CLEANING


28

Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NBA using 2018 data.

In [189]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
# def clear_data(string1):
#     if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
#         return string1
#     else:
#         return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')

def get_area(team):
    for each in list(nba_cities.index.values):
        if team in each:
            return nba_cities.loc[each, 'Metropolitan area']


# def get_nhl_data():
#     return out_df


nba_df = pd.read_csv("nba.csv")
cities = pd.read_html("wikipedia_data.html")[1] #REVISAR EL PORQUÉ DEL [1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
nba_df = nba_df[nba_df['year'] == 2018]  # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
population
cities['NBA'] = cities['NBA'].apply(lambda x: (re.split("[[]",x)[0])) #CLEANING
nba_cities = cities[['Metropolitan area', 'NBA']].set_index('NBA')
nba_cities = nba_cities.drop(['—', ''], axis=0)
nba_cities

nba_df['team'] = nba_df['team'].apply(lambda x: (re.split(("([\s][()])|([*])"),x)[0]))#CLEANING
nba_df['team']
nba_df['area'] = nba_df['team'].apply(lambda x: x.split(" ")[-1]) 
nba_df['area']

nba_df['area'] = nba_df['area'].apply(lambda x: get_area(x)) #RELACION DE EQUIPOS CON ÁREAS METROPOLITANAS
nba_df

out = []
for group, frame in nba_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = (total_wins / total_matches)
    out_dict = {
        'Area': group,
        'W': total_wins,
        'L': total_losses,
        'TM': total_matches,
        'Ratio': ratio
    }
    out.append(out_dict)

new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
new_df
out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])
out_df
population_by_region = []  # pass in metropolitan area population from cities
win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
population_by_region = out_df['Population (2016 est.)[8]']
win_loss_by_region = out_df['Ratio']
corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
corr
# len(population_by_region)
# # win_loss_by_region  


28

Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the MLB using 2018 data.

In [187]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def get_area(team):
    for each in list(mlb_cities.index.values):
        if team in each:
            return mlb_cities.loc[each, 'Metropolitan area']


mlb_df = pd.read_csv("mlb.csv")
cities = pd.read_html("wikipedia_data.html")[1] #REVISAR EL PORQUÉ DEL [1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
mlb_df = mlb_df[mlb_df['year'] == 2018]  # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
population

cities['MLB'] = cities['MLB'].apply(lambda x: (re.split("[[]",x)[0])) #CLEANING
mlb_cities = cities[['Metropolitan area', 'MLB']].set_index('MLB')
mlb_cities = mlb_cities.drop(['—', ''], axis=0)
len(mlb_cities)

# mlb_df['team'] = mlb_df['team'].apply(lambda x: (re.split(("([\s][()])|([*])"),x)[0]))#CLEANING
mlb_df['team']
mlb_df['area'] = mlb_df['team'].apply(lambda x: x.split(" ")[-1]) 
mlb_df

mlb_df['area'] = mlb_df['area'].apply(lambda x: get_area(x)) #RELACION DE EQUIPOS CON ÁREAS METROPOLITANAS
mlb_df.at[0, 'area'] = 'Boston'
mlb_df

out = []
for group, frame in mlb_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = (total_wins / total_matches)
    out_dict = {
        'Area': group,
        'W': total_wins,
        'L': total_losses,
        'TM': total_matches,
        'Ratio': ratio
    }
    out.append(out_dict)

new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
new_df
out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])
out_df
population_by_region = []  # pass in metropolitan area population from cities
win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
population_by_region = out_df['Population (2016 est.)[8]']
win_loss_by_region = out_df['Ratio']
corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
corr
# len(population_by_region)

0.1505230448710485

Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NFL using 2018 data.

In [197]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def get_area(team):
    for each in list(nfl_cities.index.values):
        if team in each:
            return nfl_cities.loc[each, 'Metropolitan area']


nfl_df = pd.read_csv("nfl.csv")
cities = pd.read_html("wikipedia_data.html")[1] #REVISAR EL PORQUÉ DEL [1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
nfl_df = nfl_df[nfl_df['year'] == 2018].drop([0, 5, 10, 15, 20, 25, 30, 35], axis=0)  # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
population

cities['NFL'] = cities['NFL'].apply(lambda x: (re.split("[[]",x)[0])) #CLEANING
nfl_cities = cities[['Metropolitan area', 'NFL']].set_index('NFL')
nfl_cities = nfl_cities.drop(['—', ''], axis=0)
len(nfl_cities)
nfl_df

nfl_df['team'] = nfl_df['team'].apply(lambda x: (re.split(("([+])|([*])"),x)[0]))#CLEANING
nfl_df['team']
nfl_df['area'] = nfl_df['team'].apply(lambda x: x.split(" ")[-1]) 
nfl_df

nfl_df['area'] = nfl_df['area'].apply(lambda x: get_area(x)) #RELACION DE EQUIPOS CON ÁREAS METROPOLITANAS
nfl_df

out = []
for group, frame in nfl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = (total_wins / total_matches)
    out_dict = {
        'Area': group,
        'W': total_wins,
        'L': total_losses,
        'TM': total_matches,
        'Ratio': ratio
    }
    out.append(out_dict)

new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
new_df
out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])
out_df
population_by_region = []  # pass in metropolitan area population from cities
win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
population_by_region = out_df['Population (2016 est.)[8]']
win_loss_by_region = out_df['Ratio']
corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
corr
# len(population_by_region)

0.004922112149349409